In [10]:
import pickle
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter

In [25]:
obs = None
with open('obsgli.json') as f:
    obs = json.load(f)

res = None
with open('res.json') as f:
    res = json.load(f)

## Obstruction

In [52]:
def n_acurracy(m, dX, dY, n=1):
    l = m.predict(pd.DataFrame(dX))
    return sum([dY[i] in l[i][:n] for i in range(len(dY))])/len(dY)

In [26]:
xObs = []
yObs = []
for item in obs:
    yObs.append(item['Obstruction'])
    del item['Obstruction']
    xObs.append(list(item.values()))

xObsTrain, xObsVal, yObsTrain, yObsVal = train_test_split(xObs, yObs, test_size=0.25, random_state=42)

In [54]:
model = pickle.load(open('modelObsClassification.pkl', 'rb'))

print(n_acurracy(model, xObsVal, yObsVal, 5))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
0.0
